In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Importando o dataset e realizando uma visualização geral/superficial inicial

In [ ]:
winequality_dataset = pd.read_csv("winequality.csv")
winequality_dataset.info()

Realizando uma análise um pouco mais detalhada a cerca de como os dados e como eles estão distribuidos

In [ ]:
winequality_dataset.describe()

In [ ]:
quantidade = {}
for i in winequality_dataset['quality']:
    if i not in quantidade:
        quantidade[i] = 1
    else:
        quantidade[i] +=1

for i in sorted(quantidade, key = quantidade.get):
    print(i, quantidade[i])

In [ ]:
#Bntender a correlação entre os dados
correlação = winequality_dataset.corr()
sns.heatmap(correlação, annot = True, fmt=".1f", cmap='coolwarm', linewidths=.6)

Limpeza do dataset

In [ ]:
#Remove a coluna chamada "Unnamed: 0".
winequality_dataset.drop('Unnamed: 0', axis=1, inplace=True)
#Substituindo os espaços entre as palavras do cabeçalho pelo underline.
winequality_dataset.columns = winequality_dataset.columns.str.replace(" ", "_")

In [ ]:
#Plotagem de um gráfico simples para analisar melhor a coluna residual_sugar.
winequality_dataset.residual_sugar.plot(kind='box', vert=False, figsize=(10,3))
#Remove os casos que possuem valores maiores que 25 (só há três valores maiores)
winequality_dataset.drop(winequality_dataset[winequality_dataset.residual_sugar > 25].index, axis=0, inplace=True)

In [ ]:
#Plotagem de um gráfico simples para analisar melhor a coluna chlorides.
winequality_dataset.chlorides.plot(kind='box', vert=False, figsize=(10,3))
#Remove os casos que possuem valores maiores que 25 (só há três valores maiores)
winequality_dataset.drop(winequality_dataset[winequality_dataset.chlorides > 0.3].index, axis=0, inplace=True)

In [ ]:
#Plotagem de um gráfico simples para analisar melhor a coluna free sulfur dioxide.
winequality_dataset.free_sulfur_dioxide.plot(kind='box', vert=False, figsize=(10,3))
#Remove os casos que possuem valores maiores que 150 (há apenas um)
winequality_dataset.drop(winequality_dataset[winequality_dataset.free_sulfur_dioxide > 120].index, axis=0, inplace=True)

In [ ]:
#Plotagem de um gráfico simples para analisar melhor a coluna total sulfur dioxide.
winequality_dataset.total_sulfur_dioxide.plot(kind='box', vert=False, figsize=(10,3))
#Remove os casos que possuem valores maiores que 250 (há apenas um)
winequality_dataset.drop(winequality_dataset[winequality_dataset.total_sulfur_dioxide > 275].index, axis=0, inplace=True)

Realizando uma classificação binária com o algoritmo de KNN (tinto ou não)

In [ ]:
#Separação do dataset
x = winequality_dataset.iloc[:, 0:11]
qualidade = winequality_dataset.iloc[:, 11]
y = winequality_dataset.iloc[:, 12]

x_treinamento, x_teste, y_treinamento, y_teste, qual_treinamento, qual_teste = train_test_split(x, y, qualidade, random_state=0, test_size=0.2)

In [ ]:
#Normalizando o dataset
normaliza = StandardScaler()

x_treinamento = normaliza.fit_transform(x_treinamento)
x_teste = normaliza.transform(x_teste)

In [ ]:
#Verificando o valor de K ideal
import math
math.sqrt(len(x_treinamento))

In [ ]:

from collections import Counter

#Euclidean Distance
def calcula_distancia(ponto1, ponto2):
    distancia = np.sqrt(np.sum((ponto1-ponto2)**2))
    return distancia
 
#Função para calcular o KNN
def predict(x_train, y_train , x_test, y_test, k):
    rotulos_y_test = []
    #Loop atraves dos pontos que precisam ser classificados
    for ponto_teste in x_test: 
        #Lista para armazenar as distancias
        point_dist = []
        #Loop atravesando cada dado do treinado
        for ponto_treino in range(len(x_train)): 
            #Calcula distancia
            distances = calcula_distancia(np.array(x_train[ponto_treino,:]), ponto_teste) 
            #Armazena as distancias na lista
            point_dist.append(distances) 

        dataframe_distancias = pd.DataFrame(data=point_dist, columns=['dist'], index=y_train.index)

        #Ordena as distancias e considera apenas as K pontos mais próximos 
        dataframe_ordenado = dataframe_distancias.sort_values(by=['dist'], axis=0)[:k]
         
        #Rastreia qual o rótulo dos K objetos mais próximos
        counter = Counter(y_train[dataframe_ordenado.index])
         
        #Pega o rótulo mais comum de todos os K vizinhos mais próximos
        prediction = counter.most_common()[0][0]
        rotulos_y_test.append(prediction)
        
    return rotulos_y_test

In [ ]:
#Realiza a predição
y_pred = predict(x_treinamento, y_treinamento, x_teste, y_teste, 71)

#Realiza o teste de acurácia
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_teste, y_pred))

In [ ]:
#Criando uma matriz de confusão simples
confusion_matrix(y_teste, y_pred)

In [ ]:
#Comparar o resultado obtido com o resultado gerado pelo algoritmo de KNN fornecido pelo sklearn
from sklearn.neighbors import KNeighborsClassifier

knn_sklearn = KNeighborsClassifier(n_neighbors=71)
knn_sklearn.fit(x_treinamento, y_treinamento)
y_pred = knn_sklearn.predict(x_teste)

print(accuracy_score(y_teste, y_pred))
print(classification_report(y_teste, y_pred))

Classificação multiclasse com o algoritmo de Random Forest (qualidade do vinho)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#Separa o dataset em treinamento e teste
x = winequality_dataset.iloc[:, 0:11]
y = winequality_dataset.iloc[:, 11]

x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, random_state=0, test_size=0.2)

In [ ]:
#Normalizando os dados
x_treinamento = normaliza.fit_transform(x_treinamento)
x_teste = normaliza.transform(x_teste)

In [ ]:
#Define função
random_forest = RandomForestClassifier()

#Define os possíveis parametros (testarei uma quantidade X de testes que combinam eles aleatóriamente, até encontrar a melhor combinação)
parametros = {
    'n_estimators': np.arange(100, 200, 10),
    "criterion": ['entropy', 'gini'],
    "max_features": ['sqrt', 'log2'],
    "bootstrap": [False, True],
    "max_depth": [10, 20, 50],
    'min_samples_leaf': np.arange(1, 5),
    'min_samples_split': np.arange(2, 11),
}

In [ ]:
#Realiza 20 combinações aleatórias entre os parametros
#busca_aleatoria = RandomizedSearchCV(estimator=random_forest, param_distributions=parametros, n_iter=20, cv=5)
#busca_aleatoria.fit(x_treinamento, y_treinamento)

#Printa o melhor conjunto de parametros encontrados
#print('Melhor modelo:' + busca_aleatoria.best_params_)

In [ ]:
#Realiza o treinamento e teste com os melhores parametros encontrados
random_forest = RandomForestClassifier(n_estimators= 100, min_samples_split= 3, min_samples_leaf= 2, max_features= 'sqrt', max_depth= 50, criterion= 'entropy', bootstrap= True)
random_forest.fit(x_treinamento, y_treinamento)
y_pred=random_forest.predict(x_teste)

In [ ]:
#Realiza o teste de acurácia
print(f'Acurácia: {accuracy_score(y_teste,y_pred):.2f}')
print(classification_report(y_teste, y_pred))



In [ ]:
#Verifica quantas quantidades de cada qualidade foi utilizada para o treinamento
quantidade = {}
for i in y_treinamento:
    if i not in quantidade:
        quantidade[i] = 1
    else:
        quantidade[i] +=1

for i in sorted(quantidade, key = quantidade.get):
    print(i, quantidade[i])

In [ ]:
#Plota um gráfico que demonstre as features mais importantes da random forest
nomes_features = winequality_dataset.iloc[:, 0:11].columns
importancia = pd.Series(data=random_forest.feature_importances_, index=nomes_features)
sns.barplot(x=importancia, y=importancia.index, orient='h').set_title('Importância de cada feature')

Aplicando algoritmo de oversampling para tentar aumentar a quantidade de dados de algumas qualidades 

In [ ]:
from imblearn.over_sampling import SMOTE

#Separa o dataset em treinamento e teste
x = winequality_dataset.iloc[:, 0:11]
y = winequality_dataset.iloc[:, 11]

smote = SMOTE(k_neighbors = 4)
x_resampleado, y_resampleado = smote.fit_resample(x, y)


In [ ]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x_resampleado, y_resampleado, random_state=0, test_size=0.2)

In [ ]:
# Treinar o modelo Random Forest usando os dados resampleados
random_forest = RandomForestClassifier(random_state=0, n_estimators= 100, min_samples_split= 3, min_samples_leaf= 2, max_features= 'sqrt', max_depth= 50, criterion= 'entropy', bootstrap= True)
random_forest.fit(x_treinamento, y_treinamento)

# Fazer previsões no conjunto de teste
y_pred = random_forest.predict(x_teste)

# Avaliar o desempenho do modelo
print("Acurácia:", accuracy_score(y_teste, y_pred))
print(classification_report(y_teste, y_pred))

In [ ]:
matriz = confusion_matrix(y_teste, y_pred)
sns.heatmap(matriz, annot=True, fmt='d', cmap='viridis')